In [ ]:
!pip install tensorflow==2.15.0 pandas numpy scikit-learn tensorboard matplotlib scikeras nltk

INFO: pip is looking at multiple versions of scikeras to determine which version is compatible with other requirements. This could take a while.


In [ ]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd


data = gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt','w') as f:
  f.write(data)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
with open('hamlet.txt','r') as file:
  text = file.read().lower()

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
print(total_words)


4818


In [ ]:
input_sequences=[]
for line in text.split('\n'):
  #print("line",line)
  tokenlist=tokenizer.texts_to_sequences([line])[0]
  #print("tokenlist",tokenlist)
  for i in range(1,len(tokenlist)):
    n_gram_sequence=tokenlist[:i+1]
    #print("n_gram_sequence",n_gram_sequence)
    input_sequences.append(n_gram_sequence)

  #print(input_sequences)


In [ ]:
max_sequence_len=max([len(x) for x in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
print(input_sequences)

[[   0    0    0 ...    0    1  687]
 [   0    0    0 ...    1  687    4]
 [   0    0    0 ...  687    4   45]
 ...
 [   0    0    0 ...    4   45 1047]
 [   0    0    0 ...   45 1047    4]
 [   0    0    0 ... 1047    4  193]]


In [ ]:
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]


In [ ]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]], dtype=int32)

In [ ]:
y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [ ]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
# model.add(Dense(total_words/2,activation='relu'))
model.add(Dense(total_words,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 lstm (LSTM)                 (None, 13, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history=model.fit(X_train,y_train,epochs=300,validation_data=(X_test,y_test),verbose=1)

Epoch 1/300
644/644 [==============================] - 24s 32ms/step - loss: 6.9287 - accuracy: 0.0331 - val_loss: 6.6947 - val_accuracy: 0.0336
Epoch 2/300
644/644 [==============================] - 20s 31ms/step - loss: 6.4787 - accuracy: 0.0393 - val_loss: 6.7672 - val_accuracy: 0.0420
Epoch 3/300
644/644 [==============================] - 20s 31ms/step - loss: 6.3407 - accuracy: 0.0455 - val_loss: 6.7980 - val_accuracy: 0.0459
Epoch 4/300
644/644 [==============================] - 20s 31ms/step - loss: 6.2000 - accuracy: 0.0502 - val_loss: 6.8151 - val_accuracy: 0.0486
Epoch 5/300
644/644 [==============================] - 20s 31ms/step - loss: 6.0644 - accuracy: 0.0541 - val_loss: 6.8530 - val_accuracy: 0.0515
Epoch 6/300
644/644 [==============================] - 20s 31ms/step - loss: 5.9237 - accuracy: 0.0602 - val_loss: 6.9219 - val_accuracy: 0.0577
Epoch 7/300
644/644 [==============================] - 20s 31ms/step - loss: 5.7971 - accuracy: 0.0670 - val_loss: 6.9985 - val_ac

In [ ]:
def generate_text(seed_text,model,max_sequence_len):
  token_list=tokenizer.texts_to_sequences([seed_text])[0]
  token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
  predicted=model.predict(token_list,verbose=0)
  predicted_word_index=np.argmax(predicted,axis=1)
  for word,index in tokenizer.word_index.items():
    if index==predicted_word_index:
      return word
  return None

In [ ]:
sample_text='i like to'
max_sequence_len=model.input_shape[1]+1
next_word=generate_text(sample_text,model,max_sequence_len)
print(next_word)

In [ ]:
model.save('hamlet_model.h5')


In [ ]:
import pickle
with open('tokenizer.pickle','wb') as handle:
  pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)